# Axisym PTST with unit ligament area

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np

In [2]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization - Force-controlled - 1N load

## Ligament of unit area

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{1} = 1$ 

In [6]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [7]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [8]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [9]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [10]:
#bond_m.interact()

In [11]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [14]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
# U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [15]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [17]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
# ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ligament area equal to 2 - Case 1: Perimiter = 2, Height = 1

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{2} = 0.5$ 

In [18]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 2.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [19]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [20]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [21]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [22]:
#bond_m.interact()

In [23]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [26]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)

In [27]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [28]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ligament area equal to 2 - Case 2: Perimiter = 1, Height = 2

Expected $\sigma_{\mathrm{T}} = \frac{Load}{Area} = \frac{1}{2} = 0.5$ 

In [29]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 2.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [30]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [31]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [32]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [33]:
#bond_m.interact()

In [34]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [36]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)

In [37]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [38]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FE discretization - Displacement-controlled - 1 mm sliding

## Ligament of area = 2

Expected $\sigma_{\mathrm{T}} = E_{\mathrm{T}} *  u_{\mathrm{T}} = 1 * 1 = 1$ 
Expected $F = \sigma_{\mathrm{T}} *  Area = 1 * 2 = 2$ 

In [39]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 2.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [40]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [41]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='u', dof=loaded_lower_dof, value = -1)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [42]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [43]:
#bond_m.interact()

In [44]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [45]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)

In [46]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [48]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# FE discretization - Applying lateral compression/tension

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{1}{1} = 1$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{1}{1} = 1$ 

In [75]:
n_y_e = 8
n_inner_x_e = 4
n_outer_x_e = 4
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_upper.mesh.I[:, -1],
    J=xd_lower.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [76]:
material_params =  dict(
     E_T=10000, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=10000, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [77]:
axisym = BCSlice(slice=xd_lower.mesh[:, 0, :, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
radial = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
                 var='u', dims=[1], value=0.001)

F = -1 / len(radial.dofs)

radial_compression = [BCDof(var='f', dof=dof, value = F) 
                                      for dof in radial.dofs ]

bc1 = [axisym, fixed_upper_support ] + radial_compression

In [78]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [79]:
#bond_m.interact()

In [80]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [81]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [82]:
time = m.hist.t
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(time, sig_pi_N[:,:,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,:,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,:,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: x and y must have same first dimension, but have shapes (11,) and (88,)

In [83]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# FE discretization - Checking stresses at Mayavi 

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{1}{1} = 1$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{1}{1} = 1$ 

In [134]:
n_y_e = 1
n_inner_x_e = 1

L_x = 1.0
R_in = 1.0 / 2 * (np.pi)

xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())

m1 = MATS3DElastic(E=1000, nu=0.3)


In [54]:
axisym = BCSlice(slice=xd_lower.mesh[:, -1, :, -1],
                 var='u', dims=[1], value=0)
fixed_support = BCSlice(slice=xd_lower.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-0.001)

# loaded_lower_dof = loaded_lower.dofs[0]
# loaded_lower_dof_2 = loaded_lower.dofs[1]

# loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -0.5)
# loaded_lower_bc_2 = BCDof(var='f', dof=loaded_lower_dof_2, value = -0.5)

# bc1 = [axisym, fixed_support, loaded_lower_bc, loaded_lower_bc_2]

loaded_lower_dof = loaded_lower.dofs

load = 1
F = load / len(loaded_lower_dof)

loaded_lower_force = [BCDof(var='f', dof=dof, value = F) 
                     for dof in loaded_lower_dof]

# bc1 = [axisym, fixed_support] + loaded_lower_force
bc1 = [axisym, fixed_support, loaded_lower] 


In [55]:
F

0.5

In [56]:
m = TStepBC(
    domains=[(xd_lower, m1),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [57]:
#bond_m.interact()

In [139]:
s.reset()
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [140]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)

In [141]:
_, (ax1) = plt.subplots(1,1)
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# Plot the punch-through curve

In [14]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [15]:
xd12.o_Eia

array([[[ 2,  3],
        [ 6,  7]],

       [[ 8,  9],
        [12, 13]]])

In [16]:
m.hist.U_t[-1][xd12.o_Eia]

array([[[-1.00001650e+00,  3.58648321e-05],
        [-9.99999850e-01, -2.63154163e-05]],

       [[ 0.00000000e+00,  7.93241176e-06],
        [-1.63545591e-05, -1.82771242e-06]]])

In [17]:
xd12.map_U_to_field(U_to[-1])[0,0]

array([ 1.00001650e+00, -2.79324204e-05])

In [18]:
F_to.shape

(2, 16)

In [19]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [20]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [195]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [44]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [45]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [46]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [47]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [48]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = -1
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
sig_pi_Tx[-1]

array([[ 3.69767199,  3.44363753],
       [ 3.44363753,  2.75394758],
       [ 2.75394758,  1.23215998],
       [ 1.23215998, -0.55897674],
       [-0.55897674, -2.55754274],
       [-2.55754274, -3.9001756 ],
       [-3.9001756 , -5.14633512],
       [-5.14633512, -6.6572502 ]])

# FE discretization - Force-controlled

In [409]:
tf_cyclic = TFCyclicNonsymmetricConstant(number_of_cycles=10, unloading_ratio=0.20)
tf_cyclic.interact()

In [388]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=37000, nu=0.18)
m2 = MATS3DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [389]:
#bond_m = Slide34(E_T=10000, E_N=10000)
# material_params =  dict(
#      E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
#      E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
# bond_m = Slide34(**material_params)

material_params =  dict(
     E_T=500, gamma_T=700, K_T=0, S_T=1., c_T=15, bartau=3, 
     E_N=1000, S_N=0.05, c_N = 2, m = 0.0, f_t=3, f_c=100, f_c0 = 75, eta=0.05 , r =12)
bond_m = Slide34(**material_params)

In [390]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-4.0)
loaded_lower_dof = loaded_lower.dofs[0]
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, time_function=tf_cyclic ,  value = 0.8 * -57.81254426110897)

bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [391]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = (1 / (36 * 2 * tf_cyclic.number_of_cycles))
s.tstep.fe_domain.serialized_subdomains

In [392]:
#bond_m.interact()

In [393]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.00(14), 
	 0.00(14), 
	 0.00(27), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(29), 
	 0.00(29), 
	 0.00(29), 
	 0.00(29), 
	 0.00(29), 
	 0.00(30), 
	 0.00(30), 
	 0.00(30), 
	 0.00(31), 
	 0.00(31), 
	 0.00(31), 
	 0.00(32), 
	 0.00(32), 
	 0.00(32), 
	 0.00(33), 
	 0.00(33), 
	 0.00(34), 
	 0.00(34), 
	 0.00(35), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(25), 
	 0.00(26), 
	 0.00(26), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(27), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(28), 
	 0.00(29), 
	 0.00(29), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14), 
	 0.00(14),

KeyboardInterrupt: 

In [394]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
# U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)

In [395]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [412]:
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
# ax1.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax1.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
ax1.legend()
idx = -1
ax_u = ax2.twinx()
ax2.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax2.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [397]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [398]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [399]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [400]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [401]:
np.max(sig_pi_Tx[:,0,0])

46.271292613929674

In [402]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = np.argmax(sig_pi_Tx[:,0,0])
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
#ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [466]:
fig, ((ax, ax_omega), (ax_creep, ax_disp)) = plt.subplots(2,2, figsize=(15,10))
idx = -1
# ax_u = ax.twinx()
lenght = int(np.floor(len(U_inner_right_t)/3))
ax.plot(-U_inner_right_t[:lenght], sig_pi_Tx[:lenght,0,0].flatten(), color='black')
ax.set_xlabel('Displacement [mm]')
ax.set_ylabel('$\sigma_{\mathrm{T}}$ [MPa]')

ax_omega.plot(-U_inner_right_t[:lenght], omega_Tx[:lenght,0,0].flatten(), color='red',  alpha=1.0, label='$\omega_T$')
ax_omega.plot(-U_inner_right_t[:lenght], omega_Nx[:lenght,0,0].flatten(), color='green',  alpha=0.98, label='omega_N')
ax_omega.set_xlabel('Displacement [mm]')
ax_omega.set_ylabel('Damage [-]')

# ax_disp.plot(-U_inner_right_t, u_pi_N[:2000,0,-1].flatten(), color='green', label='opening')
ax_disp.plot(np.arange(len(omega_Tx[:lenght:36,0,0].flatten())), omega_Tx[:lenght:36,0,0].flatten(), color='red')
ax_disp.plot(np.arange(len(omega_Nx[:lenght:36,0,0].flatten())), omega_Nx[:lenght:36,0,0].flatten(), color='green')
ax_disp.set_xlabel('Cycle [N]')
ax_disp.set_ylabel('Damage [-]')
ax_disp.set_xlim(-10,210)


ax_creep.plot(np.arange(len(omega_Tx[36:lenght:72,0,0].flatten()))*2, -U_inner_right_t[36:lenght:72], color='red')
ax_creep.plot(np.arange(len(omega_Nx[72:lenght:72,0,0].flatten()))*2, -U_inner_right_t[72:lenght:72], color='green')
ax_creep.set_xlabel('Cycle [N]')
ax_creep.set_ylabel('Displacement [mm]')
ax_creep.set_ylabel('Displacement [mm]')
ax_creep.set_xlim(-10,210)
ax_creep.set_ylim(-0.05,0.95)




ax.legend()
ax_omega.legend()
ax_disp.legend()

fig.savefig('cyclic.pdf')   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [223]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [414]:
len(U_inner_right_t)

20243